## Transfer learning 

In this notebook, I will use Transfer learning in our problem.

In [ ]:
from keras import applications
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import  Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
import os
from PIL import Image
import numpy as np

I copy $\textbf{10%}$ of the image to a new folder and this will be my new training set.

In [ ]:
train_data_dir = "../data/sample_new/train"
validation_data_dir = "../data/validation"
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
nb_validation_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])
NUM_CLASSES = 128

All the images should be resized. We will first find the smallest resolution in our datasets.

In [ ]:
smallest_train= None
for dir_path, _, files in os.walk(train_data_dir):
        for name in files:
            temp_name = os.path.join(dir_path, name)
            temp_shape = np.shape(np.array(Image.open(temp_name)))
            if smallest_train == None:
                smallest_train = temp_shape
            elif temp_shape<smallest_train:
                smallest_train = temp_shape
            else:
                continue

In [ ]:
smallest_val = None
for dir_path, _, files in os.walk(validation_data_dir):
        for name in files:
            temp_name = os.path.join(dir_path, name)
            temp_shape = np.shape(np.array(Image.open(temp_name)))
            if smallest_val == None:
                smallest_val = temp_shape
            elif temp_shape<smallest_val:
                smallest_val = temp_shape
            else:
                continue

In [ ]:
smallest = min(smallest_train, smallest_val)
img_width, img_height = smallest[0], smallest[1]

We will use the ResNet50 as model and we will fine-tune the fully connected last layer of pre-trained network.

In [ ]:
model = applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape = (img_width, img_height, 3))


Let's have a look to some of the parameters:
1. The `weights='imagenet'` parameter will load the final weights after the model trained. 
2. The `include_top=False` parameter instantiates the model without its fully connected layers.

Those layers somehow "translates" (I dunno if this the most correct word but you got my point) the convolutional information into the correct classes. Since ImageNet has 1000 different classes and our dataset 128 we don’t need any of the information in these dense layers. Instead, we will train new dense layers, the last one having 128 nodes and a softmax activation.


In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing all the layers
for layer in model.layers:
    layer.trainable = False 

In [ ]:
# # add a global spatial average pooling layer
x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(NUM_CLASSES, activation='softmax')(x)


In [ ]:
# create the full network so we can train on it
transfer_model = Model(input=model.input, output=predictions)

In [ ]:
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 1

# prepare model for fitting (loss, optimizer, etc)
transfer_model.compile(
    loss='categorical_crossentropy',  
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  
    metrics=['accuracy']  
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", k.get_value(transfer_model.optimizer.lr))

We will rescale our images

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_width, img_height),
batch_size = BATCH_SIZE,
shuffle = True,
class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
batch_size = BATCH_SIZE,
target_size = (img_width, img_height),
shuffle = True,
class_mode = "categorical")

### And we will fine tune it

In [ ]:
# Train the model 
transfer_model.fit_generator(
train_generator,
steps_per_epoch= nb_train_samples//BATCH_SIZE,
epochs = EPOCHS,
validation_data = validation_generator,
validation_steps = nb_validation_samples//BATCH_SIZE,
callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory()],
verbose=1)

 Using the `10%` of the data I achieve an accuracy of `~49%`. However, it took around 5 hours to complete just one epoch.